In [1]:
import os


os.environ['https_proxy'] = 'http://172.17.0.1:1081'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'


In [2]:
from axolotl.cli import load_cfg


cfg = load_cfg('../examples/mlpspec/llama3-8b-instruct.yaml')
cfg

/root/work/axolotl/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-11-12 14:04:35,818] [INFO] [axolotl.utils.config.models.input.check_eval_packing:993] [PID:1517532] [RANK:0] explicitly setting `eval_sample_packing` to match `sample_packing`
[2024-11-12 14:04:35,825] [INFO] [axolotl.normalize_config:183] [PID:1517532] [RANK:0] GPU memory usage baseline: 0.000GB (+1.184GB misc)


{'mlpspec': {'emb_dim': 4096,
  'inner_dim': 3072,
  'n_candidates': 5,
  'n_predict': 4,
  'top_k_tokens_per_head': [4, 3, 2, 2],
  'vocab_size': 128256,
  'decay_coefficient': 0.8},
 'medusa_logging': False,
 'medusa_scheduler': 'constant',
 'medusa_only_heads': False,
 'medusa_num_unfreeze_layers': 0,
 'medusa_distillation_regularization': 0,
 'medusa_self_distillation': False,
 'type_of_model': 'LlamaForCausalLM',
 'lisa_layers_attribute': 'model.layers',
 'wandb_project': 'mlpspec_test',
 'gradient_accumulation_steps': 4,
 'micro_batch_size': 2,
 'train_on_inputs': False,
 'group_by_length': False,
 'learning_rate': 0.0005,
 'weight_decay': 0.0,
 'optimizer': <OptimizerNames.ADAMW_BNB: 'adamw_bnb_8bit'>,
 'lr_scheduler': <SchedulerType.COSINE: 'cosine'>,
 'num_epochs': 4,
 'load_in_8bit': False,
 'load_in_4bit': False,
 'lora_dropout': 0.0,
 'qlora_sharded_model_loading': False,
 'loraplus_lr_embedding': 1e-06,
 'output_dir': './outputs/llama3-8b-instruct',
 'base_model': '/models

In [3]:
from axolotl.prompt_strategies.sharegpt import register_llama3_template


register_llama3_template(cfg.default_system_message)

In [4]:
#from axolotl.utils.models import load_model, load_tokenizer


#tokenizer = load_tokenizer(cfg)
#tokenizer

In [5]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained(cfg.base_model_config, use_fast=False)
tokenizer

PreTrainedTokenizerFast(name_or_path='/models/Meta-Llama-3-8B-Instruct', vocab_size=128000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|eot_id|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	128000: AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128004: AddedToken("<|reserved_special_token_2|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128005: AddedToken("<|

In [6]:
from axolotl.utils.models import load_model


model, peft_config = load_model(cfg, tokenizer, inference=False)
model

[2024-11-12 14:04:36,850] [INFO] [accelerate.utils.modeling.get_balanced_memory:1005] [PID:1517532] We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


/root/work/axolotl/env/lib/python3.10/site-packages/accelerate/utils/modeling.py:1381: UserWarning: Current model requires 4224 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  4.84it/s]


[2024-11-12 14:04:38,066] [INFO] [axolotl.load_model:906] [PID:1517532] [RANK:0] converting modules to torch.bfloat16 for flash attention


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    

In [7]:
model.device

device(type='cpu')

In [7]:
ids = tokenizer.encode('hello, what are you going to do', return_tensors="pt").to('cuda')
ids

tensor([[128000,  15339,     11,   1148,    527,    499,   2133,    311,    656]],
       device='cuda:0')

In [8]:
out = model.model(ids, output_hidden_states=True)
out

BaseModelOutputWithPast(last_hidden_state=tensor([[[ 4.2188e+00, -2.0410e-01, -1.8438e+00,  ..., -2.8594e+00,
           1.3594e+00,  2.8906e-01],
         [-1.4141e+00, -7.1094e-01, -5.4688e-01,  ..., -8.2031e-01,
          -1.8672e+00,  1.6484e+00],
         [-1.7812e+00, -3.1406e+00,  3.7344e+00,  ..., -1.9062e+00,
          -1.3750e+00,  2.5312e+00],
         ...,
         [-1.4531e+00,  1.1328e+00,  1.9453e+00,  ..., -1.4551e-01,
           7.1094e-01,  9.3750e-01],
         [-3.3750e+00, -9.7656e-02,  7.3047e-01,  ...,  7.8201e-04,
          -9.9609e-02,  2.2969e+00],
         [-3.5000e+00, -6.0547e-01,  4.2383e-01,  ..., -9.1797e-01,
           6.0547e-01,  2.2656e+00]]], device='cuda:0', dtype=torch.bfloat16), past_key_values=None, hidden_states=(tensor([[[-8.2970e-05,  2.5749e-04, -2.4605e-04,  ..., -3.2425e-04,
          -2.1553e-04,  4.7112e-04],
         [ 1.3123e-02, -1.8066e-02,  7.7438e-04,  ..., -7.6904e-03,
           7.2937e-03,  9.9487e-03],
         [-2.5558e-04, -4

In [9]:
out.last_hidden_state.shape

torch.Size([1, 9, 4096])

In [10]:
model.mlp_model

MLPSpeculatorPreTrainedModel(
  (speculator): MLPSpeculator(
    (emb): ModuleList(
      (0-3): 4 x Embedding(128256, 3072)
    )
    (proj): ModuleList(
      (0): Linear(in_features=4096, out_features=3072, bias=False)
      (1-3): 3 x Linear(in_features=3072, out_features=3072, bias=False)
    )
    (head): ModuleList(
      (0-3): 4 x Linear(in_features=3072, out_features=128256, bias=False)
    )
    (ln): ModuleList(
      (0-3): 4 x LayerNormParameterized()
    )
    (activation): GELU(approximate='none')
  )
)

In [11]:
speculator = model.mlp_model.speculator
len(speculator.emb)

4

In [12]:
z = speculator.emb[0](ids)
z.shape

torch.Size([1, 9, 3072])

In [13]:
state = speculator.proj[0](out.last_hidden_state)
state.shape

torch.Size([1, 9, 3072])

In [14]:
alpha = speculator.emb_weight / speculator.state_weight
alpha

17.047642905814858

In [15]:
import torch


state = torch.add(state, z, alpha=alpha)
state.shape

torch.Size([1, 9, 3072])

In [16]:
state = speculator.activation(speculator.ln[0](state))
state.shape

torch.Size([1, 9, 3072])

In [17]:
out = speculator.head[0](state)
out.shape

torch.Size([1, 9, 128256])

In [18]:
speculator.head[0].weight.shape

torch.Size([128256, 3072])

In [19]:
model.lm_head.weight.shape

torch.Size([128256, 4096])

---

In [20]:
out2 = model.forward(ids, mlpspec_return=True)
out2.shape

torch.Size([4, 1, 5, 128256])

---

In [12]:
import torch


inputs = torch.load('../inputs.pt')
inputs['input_ids'].shape, inputs['labels'].shape, inputs['position_ids'].shape, inputs['attention_mask'].shape

(torch.Size([1, 8192]),
 torch.Size([1, 8192]),
 torch.Size([1, 8192]),
 torch.Size([1, 8192]))

In [13]:
for k, v in inputs.items():
	inputs[k] = v.cpu()

inputs

{'input_ids': tensor([[128000,   2675,    527,  ..., 128001, 128001, 128001]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[-100, -100, -100,  ..., -100, -100, -100]]), 'position_ids': tensor([[0, 1, 2,  ..., 0, 0, 0]])}

In [14]:
logits = model(
	**inputs,
	mlpspec_return=True,
)
logits

RuntimeError: q must be on CUDA

In [1]:
import torch


w1 = torch.load('/root/work/axolotl/outputs/llama3-8b-instruct/pytorch_model-00001-of-00004.bin')
w1.keys()

dict_keys(['model.embed_tokens.weight', 'model.layers.0.self_attn.q_proj.weight', 'model.layers.0.self_attn.k_proj.weight', 'model.layers.0.self_attn.v_proj.weight', 'model.layers.0.self_attn.o_proj.weight', 'model.layers.0.mlp.gate_proj.weight', 'model.layers.0.mlp.up_proj.weight', 'model.layers.0.mlp.down_proj.weight', 'model.layers.0.input_layernorm.weight', 'model.layers.0.post_attention_layernorm.weight', 'model.layers.1.self_attn.q_proj.weight', 'model.layers.1.self_attn.k_proj.weight', 'model.layers.1.self_attn.v_proj.weight', 'model.layers.1.self_attn.o_proj.weight', 'model.layers.1.mlp.gate_proj.weight', 'model.layers.1.mlp.up_proj.weight', 'model.layers.1.mlp.down_proj.weight', 'model.layers.1.input_layernorm.weight', 'model.layers.1.post_attention_layernorm.weight', 'model.layers.2.self_attn.q_proj.weight', 'model.layers.2.self_attn.k_proj.weight', 'model.layers.2.self_attn.v_proj.weight', 'model.layers.2.self_attn.o_proj.weight', 'model.layers.2.mlp.gate_proj.weight', 'mode

---

In [ ]:
import os

os.environ['htts_proxy'] = 'http://172.17.0.1:1081'


In [100]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("/models/Llama-3.2-1B-vocab32k")
tokenizer

PreTrainedTokenizerFast(name_or_path='/models/Llama-3.2-1B-vocab32k', vocab_size=128000, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|eot_id|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	128000: AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128004: AddedToken("<|finetune_right_pad_id|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128005: AddedToken("<|reserved_special_token_2|>", rs

In [2]:
import torch


indices = torch.load('/root/work/chat-regen/tools/frequent-tokens.pt')
indices.shape, indices.dtype

(torch.Size([128000]), torch.int64)

In [17]:
import json


tokenizer_config = json.load(open('/models/Llama-3.2-1B-Instruct/tokenizer.json', 'r'))
tokenizer_config

{'version': '1.0',
 'truncation': None,
 'padding': None,
 'added_tokens': [{'id': 128000,
   'content': '<|begin_of_text|>',
   'single_word': False,
   'lstrip': False,
   'rstrip': False,
   'normalized': False,
   'special': True},
  {'id': 128001,
   'content': '<|end_of_text|>',
   'single_word': False,
   'lstrip': False,
   'rstrip': False,
   'normalized': False,
   'special': True},
  {'id': 128002,
   'content': '<|reserved_special_token_0|>',
   'single_word': False,
   'lstrip': False,
   'rstrip': False,
   'normalized': False,
   'special': True},
  {'id': 128003,
   'content': '<|reserved_special_token_1|>',
   'single_word': False,
   'lstrip': False,
   'rstrip': False,
   'normalized': False,
   'special': True},
  {'id': 128004,
   'content': '<|finetune_right_pad_id|>',
   'single_word': False,
   'lstrip': False,
   'rstrip': False,
   'normalized': False,
   'special': True},
  {'id': 128005,
   'content': '<|reserved_special_token_2|>',
   'single_word': False,


In [15]:
len(tokenizer_config['model']['vocab'].values())

128000

In [20]:
indices

tensor([   11,   323,   279,  ..., 79320, 79321, 79322])

In [20]:
(indices == 11).nonzero().item()

0

In [ ]:
for at in tokenizer_config['added_tokens']:
	at['id'] -= 128000 - 32000

tokenizer_config['added_tokens']

[{'id': 32000,
  'content': '<|begin_of_text|>',
  'single_word': False,
  'lstrip': False,
  'rstrip': False,
  'normalized': False,
  'special': True},
 {'id': 32001,
  'content': '<|end_of_text|>',
  'single_word': False,
  'lstrip': False,
  'rstrip': False,
  'normalized': False,
  'special': True},
 {'id': 32002,
  'content': '<|reserved_special_token_0|>',
  'single_word': False,
  'lstrip': False,
  'rstrip': False,
  'normalized': False,
  'special': True},
 {'id': 32003,
  'content': '<|reserved_special_token_1|>',
  'single_word': False,
  'lstrip': False,
  'rstrip': False,
  'normalized': False,
  'special': True},
 {'id': 32004,
  'content': '<|finetune_right_pad_id|>',
  'single_word': False,
  'lstrip': False,
  'rstrip': False,
  'normalized': False,
  'special': True},
 {'id': 32005,
  'content': '<|reserved_special_token_2|>',
  'single_word': False,
  'lstrip': False,
  'rstrip': False,
  'normalized': False,
  'special': True},
 {'id': 32006,
  'content': '<|start_

In [30]:
tokenizer_config['post_processor']['processors'][-1]['special_tokens']['<|begin_of_text|>']['ids'] = [32000]

In [31]:
pairs = list(tokenizer_config['model']['vocab'].items())
pairs

[('!', 0),
 ('"', 1),
 ('#', 2),
 ('$', 3),
 ('%', 4),
 ('&', 5),
 ("'", 6),
 ('(', 7),
 (')', 8),
 ('*', 9),
 ('+', 10),
 (',', 11),
 ('-', 12),
 ('.', 13),
 ('/', 14),
 ('0', 15),
 ('1', 16),
 ('2', 17),
 ('3', 18),
 ('4', 19),
 ('5', 20),
 ('6', 21),
 ('7', 22),
 ('8', 23),
 ('9', 24),
 (':', 25),
 (';', 26),
 ('<', 27),
 ('=', 28),
 ('>', 29),
 ('?', 30),
 ('@', 31),
 ('A', 32),
 ('B', 33),
 ('C', 34),
 ('D', 35),
 ('E', 36),
 ('F', 37),
 ('G', 38),
 ('H', 39),
 ('I', 40),
 ('J', 41),
 ('K', 42),
 ('L', 43),
 ('M', 44),
 ('N', 45),
 ('O', 46),
 ('P', 47),
 ('Q', 48),
 ('R', 49),
 ('S', 50),
 ('T', 51),
 ('U', 52),
 ('V', 53),
 ('W', 54),
 ('X', 55),
 ('Y', 56),
 ('Z', 57),
 ('[', 58),
 ('\\', 59),
 (']', 60),
 ('^', 61),
 ('_', 62),
 ('`', 63),
 ('a', 64),
 ('b', 65),
 ('c', 66),
 ('d', 67),
 ('e', 68),
 ('f', 69),
 ('g', 70),
 ('h', 71),
 ('i', 72),
 ('j', 73),
 ('k', 74),
 ('l', 75),
 ('m', 76),
 ('n', 77),
 ('o', 78),
 ('p', 79),
 ('q', 80),
 ('r', 81),
 ('s', 82),
 ('t', 83),
 

In [32]:
new_pairs = [(k, (indices == v).nonzero().item()) for k, v in pairs]
new_pairs

[('!', 792),
 ('"', 72),
 ('#', 1192),
 ('$', 1747),
 ('%', 735),
 ('&', 2495),
 ("'", 215),
 ('(', 1167),
 (')', 64),
 ('*', 162),
 ('+', 492),
 (',', 0),
 ('-', 88),
 ('.', 6),
 ('/', 106),
 ('0', 165),
 ('1', 55),
 ('2', 62),
 ('3', 43),
 ('4', 158),
 ('5', 201),
 ('6', 466),
 ('7', 420),
 ('8', 423),
 ('9', 692),
 (':', 14),
 (';', 97),
 ('<', 669),
 ('=', 1682),
 ('>', 4),
 ('?', 233),
 ('@', 885),
 ('A', 816),
 ('B', 392),
 ('C', 760),
 ('D', 980),
 ('E', 263),
 ('F', 1236),
 ('G', 1011),
 ('H', 1171),
 ('I', 942),
 ('J', 1756),
 ('K', 965),
 ('L', 1218),
 ('M', 1291),
 ('N', 470),
 ('O', 1495),
 ('P', 988),
 ('Q', 3294),
 ('R', 1295),
 ('S', 111),
 ('T', 1235),
 ('U', 635),
 ('V', 1501),
 ('W', 1117),
 ('X', 2966),
 ('Y', 2680),
 ('Z', 3154),
 ('[', 2605),
 ('\\', 2627),
 (']', 1987),
 ('^', 4718),
 ('_', 892),
 ('`', 440),
 ('a', 193),
 ('b', 21),
 ('c', 1511),
 ('d', 1345),
 ('e', 633),
 ('f', 696),
 ('g', 1190),
 ('h', 41),
 ('i', 616),
 ('j', 1665),
 ('k', 459),
 ('l', 1588)

In [ ]:
#new_pairs = [p for p in new_pairs if p[1] < 32000]
#new_pairs

[('!', 792),
 ('"', 72),
 ('#', 1192),
 ('$', 1747),
 ('%', 735),
 ('&', 2495),
 ("'", 215),
 ('(', 1167),
 (')', 64),
 ('*', 162),
 ('+', 492),
 (',', 0),
 ('-', 88),
 ('.', 6),
 ('/', 106),
 ('0', 165),
 ('1', 55),
 ('2', 62),
 ('3', 43),
 ('4', 158),
 ('5', 201),
 ('6', 466),
 ('7', 420),
 ('8', 423),
 ('9', 692),
 (':', 14),
 (';', 97),
 ('<', 669),
 ('=', 1682),
 ('>', 4),
 ('?', 233),
 ('@', 885),
 ('A', 816),
 ('B', 392),
 ('C', 760),
 ('D', 980),
 ('E', 263),
 ('F', 1236),
 ('G', 1011),
 ('H', 1171),
 ('I', 942),
 ('J', 1756),
 ('K', 965),
 ('L', 1218),
 ('M', 1291),
 ('N', 470),
 ('O', 1495),
 ('P', 988),
 ('Q', 3294),
 ('R', 1295),
 ('S', 111),
 ('T', 1235),
 ('U', 635),
 ('V', 1501),
 ('W', 1117),
 ('X', 2966),
 ('Y', 2680),
 ('Z', 3154),
 ('[', 2605),
 ('\\', 2627),
 (']', 1987),
 ('^', 4718),
 ('_', 892),
 ('`', 440),
 ('a', 193),
 ('b', 21),
 ('c', 1511),
 ('d', 1345),
 ('e', 633),
 ('f', 696),
 ('g', 1190),
 ('h', 41),
 ('i', 616),
 ('j', 1665),
 ('k', 459),
 ('l', 1588)

In [33]:
new_pairs.sort(key=lambda x: x[1])
new_pairs

[(',', 0),
 ('Ġand', 1),
 ('Ġthe', 2),
 ('li', 3),
 ('>', 4),
 ('><', 5),
 ('.', 6),
 ('Ġto', 7),
 ('Ġa', 8),
 ('p', 9),
 ('Ġof', 10),
 ('Ġin', 11),
 ('</', 12),
 ('Ġwith', 13),
 (':', 14),
 ('Ġ', 15),
 ('Ġ<', 16),
 ('Ġor', 17),
 ('ĠĊ', 18),
 ('Ġfor', 19),
 ('ĠJSON', 20),
 ('b', 21),
 ('Ġnot', 22),
 ('.</', 23),
 ('Ġis', 24),
 ('Ġ(', 25),
 ('Ġtext', 26),
 ('br', 27),
 ('.Ċ', 28),
 ('Ġjob', 29),
 ('Ġdescription', 30),
 ('Ġthat', 31),
 ('ul', 32),
 ('></', 33),
 ('Ċ', 34),
 ('Do', 35),
 ('Ġ</', 36),
 ('Ġ"', 37),
 ('Ġbe', 38),
 ('Ġas', 39),
 ('Ġan', 40),
 ('h', 41),
 ('Ġyou', 42),
 ('3', 43),
 ('Ġare', 44),
 ('Ġobject', 45),
 ('Ġcontent', 46),
 ('Ġany', 47),
 ('Ġfrom', 48),
 ('>,', 49),
 ('Ġon', 50),
 ("'s", 51),
 ('Ġinclude', 52),
 ('Ġno', 53),
 ('":', 54),
 ('1', 55),
 ('ified', 56),
 ('Ġmin', 57),
 ('Ġlanguage', 58),
 ('Ġtitle', 59),
 ('Ġour', 60),
 ('Ġoutput', 61),
 ('2', 62),
 ('Ġhave', 63),
 (')', 64),
 (')Ċ', 65),
 ('Ġoriginal', 66),
 ('ĠHTML', 67),
 ('Ġbase', 68),
 ('Ġ**', 69),
 (

In [37]:
t2i = dict(new_pairs)
t2i

{',': 0,
 'Ġand': 1,
 'Ġthe': 2,
 'li': 3,
 '>': 4,
 '><': 5,
 '.': 6,
 'Ġto': 7,
 'Ġa': 8,
 'p': 9,
 'Ġof': 10,
 'Ġin': 11,
 '</': 12,
 'Ġwith': 13,
 ':': 14,
 'Ġ': 15,
 'Ġ<': 16,
 'Ġor': 17,
 'ĠĊ': 18,
 'Ġfor': 19,
 'ĠJSON': 20,
 'b': 21,
 'Ġnot': 22,
 '.</': 23,
 'Ġis': 24,
 'Ġ(': 25,
 'Ġtext': 26,
 'br': 27,
 '.Ċ': 28,
 'Ġjob': 29,
 'Ġdescription': 30,
 'Ġthat': 31,
 'ul': 32,
 '></': 33,
 'Ċ': 34,
 'Do': 35,
 'Ġ</': 36,
 'Ġ"': 37,
 'Ġbe': 38,
 'Ġas': 39,
 'Ġan': 40,
 'h': 41,
 'Ġyou': 42,
 '3': 43,
 'Ġare': 44,
 'Ġobject': 45,
 'Ġcontent': 46,
 'Ġany': 47,
 'Ġfrom': 48,
 '>,': 49,
 'Ġon': 50,
 "'s": 51,
 'Ġinclude': 52,
 'Ġno': 53,
 '":': 54,
 '1': 55,
 'ified': 56,
 'Ġmin': 57,
 'Ġlanguage': 58,
 'Ġtitle': 59,
 'Ġour': 60,
 'Ġoutput': 61,
 '2': 62,
 'Ġhave': 63,
 ')': 64,
 ')Ċ': 65,
 'Ġoriginal': 66,
 'ĠHTML': 67,
 'Ġbase': 68,
 'Ġ**': 69,
 'Ġyour': 70,
 'Ġthis': 71,
 '"': 72,
 'Ġusing': 73,
 'Ġall': 74,
 ':</': 75,
 'Ġit': 76,
 'Ġinformation': 77,
 '<br': 78,
 ':Ċ': 79,
 'Ġnumbe

In [38]:
head32k = new_pairs[:32000]
head32k

[(',', 0),
 ('Ġand', 1),
 ('Ġthe', 2),
 ('li', 3),
 ('>', 4),
 ('><', 5),
 ('.', 6),
 ('Ġto', 7),
 ('Ġa', 8),
 ('p', 9),
 ('Ġof', 10),
 ('Ġin', 11),
 ('</', 12),
 ('Ġwith', 13),
 (':', 14),
 ('Ġ', 15),
 ('Ġ<', 16),
 ('Ġor', 17),
 ('ĠĊ', 18),
 ('Ġfor', 19),
 ('ĠJSON', 20),
 ('b', 21),
 ('Ġnot', 22),
 ('.</', 23),
 ('Ġis', 24),
 ('Ġ(', 25),
 ('Ġtext', 26),
 ('br', 27),
 ('.Ċ', 28),
 ('Ġjob', 29),
 ('Ġdescription', 30),
 ('Ġthat', 31),
 ('ul', 32),
 ('></', 33),
 ('Ċ', 34),
 ('Do', 35),
 ('Ġ</', 36),
 ('Ġ"', 37),
 ('Ġbe', 38),
 ('Ġas', 39),
 ('Ġan', 40),
 ('h', 41),
 ('Ġyou', 42),
 ('3', 43),
 ('Ġare', 44),
 ('Ġobject', 45),
 ('Ġcontent', 46),
 ('Ġany', 47),
 ('Ġfrom', 48),
 ('>,', 49),
 ('Ġon', 50),
 ("'s", 51),
 ('Ġinclude', 52),
 ('Ġno', 53),
 ('":', 54),
 ('1', 55),
 ('ified', 56),
 ('Ġmin', 57),
 ('Ġlanguage', 58),
 ('Ġtitle', 59),
 ('Ġour', 60),
 ('Ġoutput', 61),
 ('2', 62),
 ('Ġhave', 63),
 (')', 64),
 (')Ċ', 65),
 ('Ġoriginal', 66),
 ('ĠHTML', 67),
 ('Ġbase', 68),
 ('Ġ**', 69),
 (

In [43]:
from tqdm import tqdm


head = []

for k, v in tqdm(head32k):
	done = False
	for h, hi in head:
		if k in h:
			head.append((k, hi))
			done = True
			break

	if not done:
		head.append((k, v))

head

100%|██████████| 32000/32000 [00:35<00:00, 896.44it/s] 


[(',', 0),
 ('Ġand', 1),
 ('Ġthe', 2),
 ('li', 3),
 ('>', 4),
 ('><', 5),
 ('.', 6),
 ('Ġto', 7),
 ('Ġa', 1),
 ('p', 9),
 ('Ġof', 10),
 ('Ġin', 11),
 ('</', 12),
 ('Ġwith', 13),
 (':', 14),
 ('Ġ', 1),
 ('Ġ<', 16),
 ('Ġor', 17),
 ('ĠĊ', 18),
 ('Ġfor', 19),
 ('ĠJSON', 20),
 ('b', 21),
 ('Ġnot', 22),
 ('.</', 23),
 ('Ġis', 24),
 ('Ġ(', 25),
 ('Ġtext', 26),
 ('br', 27),
 ('.Ċ', 28),
 ('Ġjob', 29),
 ('Ġdescription', 30),
 ('Ġthat', 31),
 ('ul', 32),
 ('></', 33),
 ('Ċ', 18),
 ('Do', 35),
 ('Ġ</', 36),
 ('Ġ"', 37),
 ('Ġbe', 38),
 ('Ġas', 39),
 ('Ġan', 1),
 ('h', 2),
 ('Ġyou', 42),
 ('3', 43),
 ('Ġare', 44),
 ('Ġobject', 45),
 ('Ġcontent', 46),
 ('Ġany', 47),
 ('Ġfrom', 48),
 ('>,', 49),
 ('Ġon', 50),
 ("'s", 51),
 ('Ġinclude', 52),
 ('Ġno', 22),
 ('":', 54),
 ('1', 55),
 ('ified', 56),
 ('Ġmin', 57),
 ('Ġlanguage', 58),
 ('Ġtitle', 59),
 ('Ġour', 60),
 ('Ġoutput', 61),
 ('2', 62),
 ('Ġhave', 63),
 (')', 64),
 (')Ċ', 65),
 ('Ġoriginal', 66),
 ('ĠHTML', 67),
 ('Ġbase', 68),
 ('Ġ**', 69),
 ('Ġy

In [45]:
tail = []

for k, v in tqdm(new_pairs[32000:]):
	done = False
	for h, hi in head:
		if k in h:
			tail.append((k, hi))
			done = True
			break

	if not done:
		tail.append((k, v))

tail

  0%|          | 0/96000 [00:00<?, ?it/s]

100%|██████████| 96000/96000 [04:15<00:00, 375.61it/s]


[('Ġenh', 1170),
 ('shop', 3852),
 ('Ġincent', 4043),
 ('yme', 580),
 ('ĠæıĲ', 32004),
 ('ĠHF', 32005),
 ('ĠApr', 13044),
 ('ĠGuzzle', 32007),
 ('åĪĽå»º', 32008),
 ('Ġvic', 15445),
 ('Ġshading', 32010),
 ('ĠStop', 32011),
 ('ĠVec', 26986),
 ('ĠçłĶ', 32013),
 ('ĠÑģÐ¾Ð±Ð»ÑİÐ´', 32014),
 ('ĠShadow', 32015),
 ('ĠÐºÐ¾ÑĤÐ¾ÑĢÑĭÑħ', 32016),
 ('ĠOPEN', 32017),
 ('Ġspawn', 32018),
 ('Ð»Ð¸ÑĨ', 14447),
 ('åĳ¢', 32020),
 ('seek', 245),
 ('ĠDiscuss', 32022),
 ('Ġperceptions', 32023),
 ('ĠShah', 32024),
 ('/OR', 32025),
 ('NF', 11104),
 ('unks', 25661),
 ('Ġespecialmente', 32028),
 ('oten', 827),
 ('annes', 32030),
 ('Ġimmigrants', 32031),
 ('Ġmediante', 32032),
 ('ancia', 972),
 ('ĠÐºÐ¾Ð»', 32034),
 ('_Per', 32035),
 ('Ġissuing', 32036),
 ('ĠMarcos', 32037),
 ('Ġfic', 11723),
 ('ISTICS', 32039),
 ('å¯¼èĩ´', 32040),
 ('ĠExtr', 15965),
 ('Interactive', 32042),
 ('Ġprescribe', 5909),
 ('ĠXL', 32044),
 ('Ġinherit', 26471),
 ('/cal', 32046),
 ('451', 32047),
 ('æ¯ı', 32048),
 ('Ġmodo', 32049),
 ('-help',

In [82]:
resort_pairs = (head + tail)
resort_pairs.sort(key=lambda x: x[1])
resort_pairs

[(',', 0),
 ('Ġand', 1),
 ('Ġa', 1),
 ('Ġ', 1),
 ('Ġan', 1),
 ('n', 1),
 ('a', 1),
 ('and', 1),
 ('an', 1),
 ('d', 1),
 ('nd', 1),
 ('Ġthe', 2),
 ('h', 2),
 ('e', 2),
 ('t', 2),
 ('th', 2),
 ('Ġt', 2),
 ('he', 2),
 ('the', 2),
 ('Ġth', 2),
 ('li', 3),
 ('i', 3),
 ('l', 3),
 ('>', 4),
 ('><', 5),
 ('<', 5),
 ('.', 6),
 ('Ġto', 7),
 ('o', 7),
 ('to', 7),
 ('p', 9),
 ('Ġof', 10),
 ('f', 10),
 ('Ġo', 10),
 ('of', 10),
 ('Ġin', 11),
 ('in', 11),
 ('Ġi', 11),
 ('</', 12),
 ('/', 12),
 ('Ġwith', 13),
 ('w', 13),
 ('it', 13),
 ('Ġw', 13),
 ('with', 13),
 ('ith', 13),
 ('wi', 13),
 ('Ġwit', 13),
 ('Ġwi', 13),
 ('wit', 13),
 (':', 14),
 ('Ġ<', 16),
 ('Ġor', 17),
 ('or', 17),
 ('r', 17),
 ('ĠĊ', 18),
 ('Ċ', 18),
 ('Ġfor', 19),
 ('Ġf', 19),
 ('for', 19),
 ('fo', 19),
 ('Ġfo', 19),
 ('ĠJSON', 20),
 ('S', 20),
 ('N', 20),
 ('O', 20),
 ('ĠJ', 20),
 ('J', 20),
 ('ON', 20),
 ('JS', 20),
 ('SO', 20),
 ('ĠJS', 20),
 ('JSON', 20),
 ('SON', 20),
 ('b', 21),
 ('Ġnot', 22),
 ('Ġno', 22),
 ('no', 22),
 ('Ġn',

In [83]:
resort_pairs[31950:32050]

[('å£', 18397),
 ('ĠIv', 18398),
 ('éłħ', 18400),
 ('éł', 18400),
 ('Ġdecimal', 18402),
 ('decimal', 18402),
 ('ecimal', 18402),
 ('cimal', 18402),
 ('Ġê°ľë°ľ', 18403),
 ('ë°ľ', 18403),
 ('ê°ľë°ľ', 18403),
 ('Ġportions', 18404),
 ('Ġstatue', 18405),
 ('Ġattracted', 18407),
 ('Ġinterruptions', 18409),
 ('Ġinterrupt', 18409),
 ('interrupt', 18409),
 ('Ġinterruption', 18409),
 ('errupt', 18409),
 ('.db', 18410),
 ('ĠAmy', 18412),
 ('Amy', 18412),
 ('-pressure', 18413),
 ('-pres', 18413),
 ('åĽ³', 18416),
 ('Ġassault', 18418),
 ('Ġsemana', 18419),
 ('Ġdiving', 18423),
 ('ĠTransparent', 18424),
 ('Transparent', 18424),
 ('ĠCampaign', 18425),
 ('Campaign', 18425),
 ('Ġkostenlose', 18426),
 ('Ġkostenlos', 18426),
 ('éĩĮ', 18427),
 ('ĩĮ', 18427),
 ('Ġkok', 18428),
 ('kok', 18428),
 ('Ġduplicates', 18429),
 ('Ġduplicate', 18429),
 ('Ġdup', 18429),
 ('dup', 18429),
 ('Ġduplic', 18429),
 ('uplicates', 18429),
 ('uplicate', 18429),
 ('duplicate', 18429),
 ('uplic', 18429),
 ('åī²', 18430),
 ('Ġove

In [84]:
in_pairs = [(p[0], i) for i, p in enumerate(resort_pairs[:32000])]
in_pairs

[(',', 0),
 ('Ġand', 1),
 ('Ġa', 2),
 ('Ġ', 3),
 ('Ġan', 4),
 ('n', 5),
 ('a', 6),
 ('and', 7),
 ('an', 8),
 ('d', 9),
 ('nd', 10),
 ('Ġthe', 11),
 ('h', 12),
 ('e', 13),
 ('t', 14),
 ('th', 15),
 ('Ġt', 16),
 ('he', 17),
 ('the', 18),
 ('Ġth', 19),
 ('li', 20),
 ('i', 21),
 ('l', 22),
 ('>', 23),
 ('><', 24),
 ('<', 25),
 ('.', 26),
 ('Ġto', 27),
 ('o', 28),
 ('to', 29),
 ('p', 30),
 ('Ġof', 31),
 ('f', 32),
 ('Ġo', 33),
 ('of', 34),
 ('Ġin', 35),
 ('in', 36),
 ('Ġi', 37),
 ('</', 38),
 ('/', 39),
 ('Ġwith', 40),
 ('w', 41),
 ('it', 42),
 ('Ġw', 43),
 ('with', 44),
 ('ith', 45),
 ('wi', 46),
 ('Ġwit', 47),
 ('Ġwi', 48),
 ('wit', 49),
 (':', 50),
 ('Ġ<', 51),
 ('Ġor', 52),
 ('or', 53),
 ('r', 54),
 ('ĠĊ', 55),
 ('Ċ', 56),
 ('Ġfor', 57),
 ('Ġf', 58),
 ('for', 59),
 ('fo', 60),
 ('Ġfo', 61),
 ('ĠJSON', 62),
 ('S', 63),
 ('N', 64),
 ('O', 65),
 ('ĠJ', 66),
 ('J', 67),
 ('ON', 68),
 ('JS', 69),
 ('SO', 70),
 ('ĠJS', 71),
 ('JSON', 72),
 ('SON', 73),
 ('b', 74),
 ('Ġnot', 75),
 ('Ġno', 76),

In [85]:
tokenizer_config['model']['vocab'] = dict(in_pairs)
tokenizer_config['model']['vocab']

{',': 0,
 'Ġand': 1,
 'Ġa': 2,
 'Ġ': 3,
 'Ġan': 4,
 'n': 5,
 'a': 6,
 'and': 7,
 'an': 8,
 'd': 9,
 'nd': 10,
 'Ġthe': 11,
 'h': 12,
 'e': 13,
 't': 14,
 'th': 15,
 'Ġt': 16,
 'he': 17,
 'the': 18,
 'Ġth': 19,
 'li': 20,
 'i': 21,
 'l': 22,
 '>': 23,
 '><': 24,
 '<': 25,
 '.': 26,
 'Ġto': 27,
 'o': 28,
 'to': 29,
 'p': 30,
 'Ġof': 31,
 'f': 32,
 'Ġo': 33,
 'of': 34,
 'Ġin': 35,
 'in': 36,
 'Ġi': 37,
 '</': 38,
 '/': 39,
 'Ġwith': 40,
 'w': 41,
 'it': 42,
 'Ġw': 43,
 'with': 44,
 'ith': 45,
 'wi': 46,
 'Ġwit': 47,
 'Ġwi': 48,
 'wit': 49,
 ':': 50,
 'Ġ<': 51,
 'Ġor': 52,
 'or': 53,
 'r': 54,
 'ĠĊ': 55,
 'Ċ': 56,
 'Ġfor': 57,
 'Ġf': 58,
 'for': 59,
 'fo': 60,
 'Ġfo': 61,
 'ĠJSON': 62,
 'S': 63,
 'N': 64,
 'O': 65,
 'ĠJ': 66,
 'J': 67,
 'ON': 68,
 'JS': 69,
 'SO': 70,
 'ĠJS': 71,
 'JSON': 72,
 'SON': 73,
 'b': 74,
 'Ġnot': 75,
 'Ġno': 76,
 'no': 77,
 'Ġn': 78,
 'ot': 79,
 'not': 80,
 '.</': 81,
 '.<': 82,
 'Ġis': 83,
 's': 84,
 'is': 85,
 'Ġ(': 86,
 '(': 87,
 'Ġtext': 88,
 'x': 89,
 'te': 

In [86]:
vocab = tokenizer_config['model']['vocab']

In [87]:
tokenizer_config['model']['merges']

['Ġ Ġ',
 'Ġ ĠĠĠ',
 'ĠĠ ĠĠ',
 'ĠĠĠ Ġ',
 'i n',
 'Ġ t',
 'Ġ ĠĠĠĠĠĠĠ',
 'ĠĠ ĠĠĠĠĠĠ',
 'ĠĠĠĠ ĠĠĠĠ',
 'ĠĠĠ ĠĠĠĠĠ',
 'ĠĠĠĠĠĠĠ Ġ',
 'ĠĠĠĠĠ ĠĠĠ',
 'ĠĠĠĠĠĠ ĠĠ',
 'e r',
 'Ġ ĠĠ',
 'ĠĠ Ġ',
 'o n',
 'Ġ a',
 'r e',
 'a t',
 's t',
 'e n',
 'o r',
 'Ġ th',
 'Ġt h',
 'Ċ Ċ',
 'Ġ c',
 'l e',
 'Ġ s',
 'i t',
 'a n',
 'a r',
 'a l',
 'Ġ the',
 'Ġt he',
 'Ġth e',
 '; Ċ',
 'Ġ p',
 'Ġ f',
 'o u',
 'Ġ =',
 'i s',
 'Ġ ĠĠĠĠĠĠ',
 'ĠĠ ĠĠĠĠĠ',
 'ĠĠĠĠ ĠĠĠ',
 'ĠĠĠ ĠĠĠĠ',
 'ĠĠĠĠĠ ĠĠ',
 'ĠĠĠĠĠĠ Ġ',
 'i ng',
 'in g',
 'e s',
 'Ġ w',
 'i on',
 'io n',
 'e d',
 'i c',
 'Ġ b',
 'Ġ d',
 'e t',
 'Ġ m',
 'Ġ o',
 'ĉ ĉ',
 'r o',
 'a s',
 'e l',
 'c t',
 'n d',
 'Ġ in',
 'Ġi n',
 'Ġ h',
 'e nt',
 'en t',
 'i d',
 'Ġ n',
 'a m',
 'Ġ ĠĠĠĠĠĠĠĠĠĠ',
 'ĠĠ ĠĠĠĠĠĠĠĠĠ',
 'ĠĠĠĠ ĠĠĠĠĠĠĠ',
 'ĠĠĠĠĠĠĠĠ ĠĠĠ',
 'ĠĠĠ ĠĠĠĠĠĠĠĠ',
 'ĠĠĠĠĠĠĠ ĠĠĠĠ',
 'ĠĠĠĠĠ ĠĠĠĠĠĠ',
 'ĠĠĠĠĠĠĠĠĠ ĠĠ',
 'ĠĠĠĠĠĠ ĠĠĠĠĠ',
 'ĠĠĠĠĠĠĠĠĠĠ Ġ',
 'Ġ to',
 'Ġt o',
 'Ġ re',
 'Ġr e',
 '- -',
 'Ġ {',
 'Ġ of',
 'Ġo f',
 'o m',
 ') ;Ċ',
 '); Ċ',
 'i m',
 'Ġ (',
 'i l',
 '/ /',
 'Ġ an

In [94]:
def validate_merge (m):
	t1, t2 = m.split(' ')
	#return t1 in vocab and t2 in vocab and (t1 + t2) in vocab
	return (t1 + t2) in vocab

new_merges = [m for m in tokenizer_config['model']['merges'] if validate_merge(m)]
len(new_merges)

78556

In [95]:
tokenizer_config['model']['merges'] = new_merges

In [96]:
json.dump(tokenizer_config, open('/models/Llama-3.2-1B-vocab32k/tokenizer.json', 'w'), indent=2, ensure_ascii=False)

---

In [98]:
tokenizer.encode('hello world!')

[32000, 25943, 3285, 2150]

In [ ]:
tokenizer.encode('你好')

[32000, 4144, 7421]

In [102]:
tokenizer.encode('天地玄黄宇宙洪荒')

[32000, 14894, 8583, 90420, 94330, 34068, 32977, 95384, 93895]

---
## prune model weights

In [2]:
from safetensors import safe_open


with safe_open("/models/Llama-3.2-1B-Instruct/model.safetensors", 'pt', device='cpu') as f:
	for key in f.keys():
		print(key, f.get_tensor(key).shape)


model.embed_tokens.weight torch.Size([128256, 2048])
model.layers.0.input_layernorm.weight torch.Size([2048])
model.layers.0.mlp.down_proj.weight torch.Size([2048, 8192])
model.layers.0.mlp.gate_proj.weight torch.Size([8192, 2048])
model.layers.0.mlp.up_proj.weight torch.Size([8192, 2048])
model.layers.0.post_attention_layernorm.weight torch.Size([2048])
model.layers.0.self_attn.k_proj.weight torch.Size([512, 2048])
model.layers.0.self_attn.o_proj.weight torch.Size([2048, 2048])
model.layers.0.self_attn.q_proj.weight torch.Size([2048, 2048])
model.layers.0.self_attn.v_proj.weight torch.Size([512, 2048])
model.layers.1.input_layernorm.weight torch.Size([2048])
model.layers.1.mlp.down_proj.weight torch.Size([2048, 8192])
model.layers.1.mlp.gate_proj.weight torch.Size([8192, 2048])
model.layers.1.mlp.up_proj.weight torch.Size([8192, 2048])
model.layers.1.post_attention_layernorm.weight torch.Size([2048])
model.layers.1.self_attn.k_proj.weight torch.Size([512, 2048])
model.layers.1.self_at

In [4]:
import torch


indices = torch.load('/root/work/chat-regen/tools/frequent-tokens.pt')
indices.shape, indices

(torch.Size([128000]),
 tensor([   11,   323,   279,  ..., 79320, 79321, 79322]))

In [5]:
with safe_open("/models/Llama-3.2-1B-Instruct/model.safetensors", 'pt', device='cpu') as f:
	embed_tokens = f.get_tensor('model.embed_tokens.weight')

embed_tokens.shape

torch.Size([128256, 2048])

In [8]:
w4 = embed_tokens[indices[:4]]

w4[0] == embed_tokens[11]

tensor([True, True, True,  ..., True, True, True])

In [13]:
tail_indices = torch.arange(256).long() + 128000
new_indices = torch.cat([indices[:32000], tail_indices], dim=0)
new_indices.shape

torch.Size([32256])

In [14]:
new_embed_tokens = embed_tokens[new_indices]
new_embed_tokens.shape

torch.Size([32256, 2048])

In [21]:
state = dict()

with safe_open("/models/Llama-3.2-1B-Instruct/model.safetensors", 'pt', device='cpu') as f:
	for key in f.keys():
		if key == 'model.embed_tokens.weight':
			state[key] = new_embed_tokens
		else:
			state[key] = f.get_tensor(key)

	print(f.metadata())

state

{'format': 'pt'}


{'model.embed_tokens.weight': tensor([[ 2.7008e-03, -2.9419e-02,  4.8340e-02,  ..., -1.4709e-02,
          -1.8066e-02, -3.8574e-02],
         [-1.8387e-03,  2.0630e-02,  4.3701e-02,  ..., -1.9653e-02,
           1.6479e-03,  6.8359e-03],
         [-8.0566e-03, -1.5015e-02,  2.6001e-02,  ..., -1.1749e-03,
          -3.6621e-03, -1.5869e-02],
         ...,
         [-1.0580e-06,  1.0620e-02, -1.9043e-02,  ...,  1.3885e-03,
          -1.7700e-03,  9.7046e-03],
         [-1.3635e-06,  1.0620e-02, -1.9043e-02,  ...,  1.3885e-03,
          -1.7700e-03,  9.7046e-03],
         [-1.1921e-06,  1.0620e-02, -1.9043e-02,  ...,  1.3885e-03,
          -1.7700e-03,  9.7046e-03]], dtype=torch.bfloat16),
 'model.layers.0.input_layernorm.weight': tensor([0.1543, 0.1826, 0.2559,  ..., 0.2168, 0.2080, 0.1494],
        dtype=torch.bfloat16),
 'model.layers.0.mlp.down_proj.weight': tensor([[-0.0269,  0.0176, -0.0003,  ..., -0.0194,  0.0115,  0.0058],
         [ 0.0156, -0.0057,  0.0072,  ..., -0.0233,  0.00

In [22]:
from safetensors.torch import save_file


save_file(state, "/models/Llama-3.2-1B-vocab32k/model.safetensors", metadata={'format': 'pt'})

In [17]:
with safe_open("/models/Llama-3.2-1B-vocab32k/model.safetensors", 'pt', device='cpu') as f:
	for key in f.keys():
		print(key, f.get_tensor(key).shape)


model.embed_tokens.weight torch.Size([32256, 2048])
model.layers.0.input_layernorm.weight torch.Size([2048])
model.layers.0.mlp.down_proj.weight torch.Size([2048, 8192])
model.layers.0.mlp.gate_proj.weight torch.Size([8192, 2048])
model.layers.0.mlp.up_proj.weight torch.Size([8192, 2048])
model.layers.0.post_attention_layernorm.weight torch.Size([2048])
model.layers.0.self_attn.k_proj.weight torch.Size([512, 2048])
model.layers.0.self_attn.o_proj.weight torch.Size([2048, 2048])
model.layers.0.self_attn.q_proj.weight torch.Size([2048, 2048])
model.layers.0.self_attn.v_proj.weight torch.Size([512, 2048])
model.layers.1.input_layernorm.weight torch.Size([2048])
model.layers.1.mlp.down_proj.weight torch.Size([2048, 8192])
model.layers.1.mlp.gate_proj.weight torch.Size([8192, 2048])
model.layers.1.mlp.up_proj.weight torch.Size([8192, 2048])
model.layers.1.post_attention_layernorm.weight torch.Size([2048])
model.layers.1.self_attn.k_proj.weight torch.Size([512, 2048])
model.layers.1.self_att

---

In [35]:
from transformers import AutoModelForCausalLM, AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("/models/Llama-3.2-1B-vocab32k")
model = AutoModelForCausalLM.from_pretrained("/models/Llama-3.2-1B-vocab32k")
tokenizer, model

(PreTrainedTokenizerFast(name_or_path='/models/Llama-3.2-1B-vocab32k', vocab_size=32000, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|eot_id|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
 	32000: AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	32001: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	32002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	32003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	32004: AddedToken("<|finetune_right_pad_id|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	32005: AddedToken("<|reserved_special_token_2|>", rs

In [44]:
ids = tokenizer.encode("Hello, ", return_tensors="pt")
ids

tensor([[32000,  6214,     0,    15]])

In [45]:
output = model.generate(ids)
output

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


tensor([[32000,  6214,     0,    15,   192,  1678,   620,     7,   555,     8,
            83,  1489,    19,     8,   231,    31, 25327,  6187,  1914,   872]])

In [46]:
tokenizer.decode(output[0])

'<|begin_of_text|>Hello,  I am looking to create a new website for a business that sells eco-friendly products'

In [47]:
import torch
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model='/models/Llama-3.2-1B-vocab32k',
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]
outputs = pipe(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'role': 'assistant', 'content': 'Yer lookin\' fer me, matey? I be the swashbucklin\' pirlot, here to serve up the scurvy dog answers yerself. Me name be "Blackbeak Bill", the greatest answer-giver on the seven oceans... er, I mean, the internet. Me knowledge be vast, me wisdom be boundless, and me responses be as smooth as a chest overflowin\' with gold coins. So hoist the colors, me hearty, and ask me any question that be on yers mind!'}


In [5]:
import torch
from transformers import AutoTokenizer


tok1 = AutoTokenizer.from_pretrained('/models/Llama-3.2-1B-Instruct')
tok2 = AutoTokenizer.from_pretrained('/models/Llama-3.2-1B-vocab32k')
indices = torch.load('/root/work/chat-regen/tools/frequent-tokens.pt')

tail_indices = torch.arange(256, dtype=torch.int64) + 128000
total_indices = torch.cat([indices[:32000], tail_indices])

ids1 = tok2.encode('You are a pirate chatbot who always responds in pirate speak!')
ids2 = [total_indices[id] for id in ids1]
tok1.decode(ids2)

'<|begin_of_text|>You are a pirate chatbot who always responds in pirate speak!'

In [6]:
torch.save(total_indices, '/models/Llama-3.2-1B-vocab32k/token_indices.pt')

In [7]:
inv_indices = [(indices == id).nonzero().item() for id in range(128000)]
inv_indices

[792,
 72,
 1192,
 1747,
 735,
 2495,
 215,
 1167,
 64,
 162,
 492,
 0,
 88,
 6,
 106,
 165,
 55,
 62,
 43,
 158,
 201,
 466,
 420,
 423,
 692,
 14,
 97,
 669,
 1682,
 4,
 233,
 885,
 816,
 392,
 760,
 980,
 263,
 1236,
 1011,
 1171,
 942,
 1756,
 965,
 1218,
 1291,
 470,
 1495,
 988,
 3294,
 1295,
 111,
 1235,
 635,
 1501,
 1117,
 2966,
 2680,
 3154,
 2605,
 2627,
 1987,
 4718,
 892,
 440,
 193,
 21,
 1511,
 1345,
 633,
 696,
 1190,
 41,
 616,
 1665,
 459,
 1588,
 216,
 134,
 948,
 9,
 5294,
 1206,
 85,
 1197,
 779,
 1210,
 945,
 1137,
 1313,
 686,
 5270,
 3140,
 3325,
 2356,
 4512,
 6494,
 10613,
 3920,
 13793,
 11606,
 8374,
 10232,
 9047,
 9496,
 11593,
 13533,
 7874,
 9514,
 10280,
 9881,
 12873,
 14653,
 4652,
 14242,
 11583,
 7504,
 14274,
 12825,
 12020,
 17719,
 9979,
 17526,
 10286,
 8512,
 81591,
 81540,
 81841,
 81730,
 27969,
 33841,
 35202,
 24179,
 21830,
 33620,
 20255,
 45398,
 32281,
 44240,
 79227,
 25972,
 83571,
 31908,
 42913,
 67836,
 44347,
 42760,
 44393,
 1248

In [8]:
inv_indices_tensor = torch.tensor(inv_indices)
inv_indices_tensor[inv_indices_tensor >= 32000] = -1
inv_indices_tensor

tensor([ 792,   72, 1192,  ...,   -1,   -1,   -1])

In [9]:
inv_indices_tensor = torch.cat([inv_indices_tensor, torch.arange(256) + 32000])
inv_indices_tensor.shape, inv_indices_tensor

(torch.Size([128256]),
 tensor([  792,    72,  1192,  ..., 32253, 32254, 32255]))

In [11]:
torch.save(inv_indices_tensor, '/models/Llama-3.2-1B-vocab32k/inv_token_indices.pt')

In [16]:
import os

os.environ['https_proxy'] = 'http://172.17.0.1:1081'

In [17]:
from huggingface_hub import hf_hub_download


indices_path = hf_hub_download(repo_id='k-l-lambda/Llama-3.2-1B-vocab32k', filename='token_indices.pt')
indices_path

'/root/.cache/huggingface/hub/models--k-l-lambda--Llama-3.2-1B-vocab32k/snapshots/8392603156385cf71e76d565843efa856c25c8e0/token_indices.pt'